In [1]:
import cv2
from ultralytics import YOLO    #  from ultralytics import 
from matplotlib import pyplot as plt
import pickle

from skimage.transform import resize
import numpy as np
import cv2


In [2]:
mask = "../Data_Used_in_this_project/mask_1920_1080.png"
video = "../Data_Used_in_this_project/parking_1920_1080.mp4"

mask = cv2.imread(mask ,0)
cap = cv2.VideoCapture(video)

print(mask.shape)

(1080, 1920)


In [3]:
def get_spot_location(Commponents):
    [total_label , label_id , values ,centroid] = Commponents
    coeff = 1
    slots = []
    for i in range(1,total_label):
        x1 = int(values[i,cv2.CC_STAT_LEFT]*coeff)
        y1 = int(values[i,cv2.CC_STAT_TOP]*coeff)
        w = int(values[i,cv2.CC_STAT_WIDTH]*coeff)
        h = int(values[i,cv2.CC_STAT_HEIGHT]*coeff)
        slots.append([x1,y1,w,h])
    return slots

connected_components = cv2.connectedComponentsWithStats(mask,4,cv2.CV_32S)
spots = get_spot_location(connected_components)

In [4]:

MODEL = pickle.load(open("../Models/model.p", "rb"))
def empty_or_not(spot_bgr):

    flat_data = []

    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return True
    else:
        return False


c:\Users\HamzaFayaz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
def calc_diff(img1,img2):
    return np.abs(np.mean(img1) - np.mean(img2))


spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30

In [6]:
while ret:
    ret, frame = cap.read()

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_index, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]
            diffs[spot_index] = calc_diff(spot_crop, previous_frame[y1:y1+h, x1:x1+w, :])
            # print([diffs[j] for j in np.argsort(diffs)][::-1])
            # plt.hist([diffs[j] for j in np.argsort(diffs)][::-1])

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]

        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]
            spot_status = empty_or_not(spot_crop)
            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.putText(frame, f"Available spots {str(sum(spots_status))} / {str(len(spots_status))}", (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(25) & 0xff == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()